In [45]:
import torch
from d2l import torch as d2l

In [46]:
class SGD(d2l.HyperParameters):
    def __init__(self, params, lr):
        self.save_hyperparameters()

    def step(self):
        for param in self.params:
            param -= self.lr * param.grad

    def zero_grad(self):
        for param in self.params:
            if param.grad is not None:
                param.grad.zero_()

In [47]:
class LinearRegressionScratch(d2l.Module):
    def __init__(self, num_inputs, lr,  sigma=0.01):
        super().__init__()
        self.save_hyperparameters()

        self.w = torch.normal(0, sigma, (num_inputs, 1), requires_grad=True)
        self.b = torch.zeros(1, requires_grad=True)

    def forward(self, X):
        return torch.matmul(X, self.w) + self.b

    def loss(self, y_hat, y):
        l = ((y_hat - y) ** 2) / 2
        return l.mean()

    def configure_optimizers(self):
        return SGD([self.w, self.b], self.lr)

In [49]:
class RalphTrainer(d2l.Trainer):

    def fit_epoch(self):
        self.model.train()
        for batch in self.train_dataloader:
            loss = self.model.training_step(batch)
            self.optim.zero_grad()
            with torch.no_grad():
                loss.backward()
                if self.gradient_clip_val > 0:
                    self.clip_gradients(self.gradient_clip_val, self.model)
                self.optim.step()

            self.train_batch_idx += 1

        if self.val_dataloader is None:
            return

        self.model.eval()
        for batch in self.val_dataloader:
            with torch.no_grad():
                self.model.validation_step(batch)

            self.val_batch_idx += 1

In [52]:
model = LinearRegressionScratch(2, lr=0.03)
data = d2l.SyntheticRegressionData(w=torch.tensor([2, -3.4]), b=4.2)

In [55]:
trainer = RalphTrainer(1)

In [56]:
trainer.fit(model, data)

ValueError: optimizer got an empty parameter list